# Fast Risk Parity Prototyping
This notebook aims to provide a straightforward and simpler access to the risk parity portfolio construction. The main idea is to provide a simple and fast way to prototype risk parity portfolios. Most of the code is inspired by `run_backtest.py`, which is considered the entry point of this package.

In [1]:
from portfolio_optimization.data_processing import *
from portfolio_optimization.data_collection import *
from tokens.get_assets import *
import numpy as np
from portfolio_optimization.optimization.risk_parity import RiskParity

from portfolio_optimization.portfolio.Portfolio import Portfolio
from portfolio_optimization.backtesting.Backtesting import Backtest
from datetime import timedelta
from dateutil.relativedelta import relativedelta

In [2]:
# Choose the asset class and the assets
asset_list = get_tickers()
asset_class = "high_risk_tickers"

High risk tickers: 12
Medium risk tickers: 60
Low risk tickers: 8


In [3]:
# Data processing
_df = get_historical_prices_for_assets(
    asset_list[asset_class],
    time_range=timedelta(days=365 * 1 + 120),  # 1 years
    interested_columns=["ReferenceRate", "CapMrktEstUSD"],
)

# Filter out all columns containing `_` in their name
df = _df.loc[:, ~_df.columns.str.contains("_")]

# Get all the market caps
mcaps = _df.loc[:, _df.columns.str.contains("CapMrktEstUSD")]
mcaps.columns = mcaps.columns.str.replace("_CapMrktEstUSD", "")
mcaps.replace(np.nan, 0, inplace=True)

start_date_portfolio = df.index[0] + relativedelta(days=120)

/Users/arguiot/Developer/Pyratz/portfolio/portfolio_optimization/data_collection/get_crypto_price_range.py:41: DtypeWarning: Columns (146) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/d2/1qh2h9ys19z3375prb_0cb8w0000gn/T/ipykernel_22317/3686626402.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcaps.replace(np.nan, 0, inplace=True)


## Setting up the backtesting parameters

In [4]:
initial_bid = 1000 # We start with 1000 USD

max_weight = {"*": 1.0} # Can be: {'*': 1.0} or {'*': 0.5, 'BTC': 0.15}

min_weight = { '*': 0.0} # Can be: {'*': 0.0} or {'*': 0.5, 'BTC': 0.05}

budget = {} # Can be: {'uni': 1.2} for extra 20% for UNI

yield_data = pd.Series()
for asset in asset_list[asset_class]:
    yield_data[asset] = 0.075

weight_threshold = 0.01

lambda_var = 0.1
lambda_u = 0.1

rebalance_frequency = "1W"
adjust_holdings = True # Wether to adjust the holdings daily to obtain the target weights

## Portfolio Construction & backtesting
We build the portfolio for backtesting

In [5]:
portfolio_parity = Portfolio(
    base_value=initial_bid,
    initial_prices=df.loc[:start_date_portfolio],
    optimiser=RiskParity,
    max_weight=max_weight,
    min_weight=min_weight,
    weight_threshold=weight_threshold,
    budget=budget,
    lambda_var=lambda_var,
    lambda_u=lambda_u,
)

backtest = Backtest(
    portfolios={
        "Risk Parity": portfolio_parity,
    },
    start_date=start_date_portfolio,
    end_date=df.index[-1],
    rebalance_frequency=rebalance_frequency,
    adjust_holdings=adjust_holdings,
    data=df,
    mcaps=mcaps,
    asset_class=asset_class,
)

#### Saving the data

In [6]:
perfs = backtest.run_backtest(
    look_back_period=120,
    look_back_unit="D",
    yield_data=yield_data,
)

# Check if the directory exists, if not, create it
if not os.path.exists(f"./out/{rebalance_frequency}_parity/"):
    os.makedirs(f"./out/{rebalance_frequency}_parity/")

backtest.export_results(
    perfs, f"./out/{rebalance_frequency}_parity/", f"backtest_results_{asset_class}.xlsx"
)

/Users/arguiot/Developer/Pyratz/portfolio/portfolio_optimization/backtesting/Backtesting.py:290: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value["Daily Return"] = daily_return


: 